# Dublin Networkx

### Analysing Dublin's road network using NetworkX

This Notebook provides an overview of importing the Shapefile data
into the NetworkX Python library

### Exploration

Begin by importing the modules and required GRASS data

In [1]:
import networkx as nx # use patched fork from github.com/ruaridhw/networkx/tree/write-shp-dev
%matplotlib inline

In [2]:
%time G = nx.read_shp('../2_grass_data_output/connections_shp/connections.shp')

CPU times: user 2min 45s, sys: 1.06 s, total: 2min 46s
Wall time: 2min 47s


Examine the number of nodes and edges:

In [3]:
print(nx.info(G))

Name: 
Type: DiGraph
Number of nodes: 85119
Number of edges: 95867
Average in degree:   1.1263
Average out degree:   1.1263


What is the degree of each node?

In [4]:
nx.degree_histogram(G)

[0, 21221, 24358, 36432, 3043, 61, 4]

Is the road network strongly connected as expected?

In [5]:
nx.is_strongly_connected(G)

False

In [6]:
nx.number_weakly_connected_components(G)

36

Print out the first 10 edges:

In [7]:
for i, edge in enumerate(G.edges()):
    if i == 10:
        break
    print(edge)

((317995.5708697437, 245878.08877053304), (317973.5650240276, 245907.60584958008))
((317995.5708697437, 245878.08877053304), (318026.7365888019, 245907.81662953694))
((317973.5650240276, 245907.60584958008), (318382.4449278968, 246117.12593351814))
((317960.94469078025, 245853.84258207353), (317941.54076457256, 245885.65162972905))
((317960.94469078025, 245853.84258207353), (317986.6586180811, 245862.27751578222))
((317941.54076457256, 245885.65162972905), (317942.0665525985, 245891.2323251177))
((317894.5420938695, 245824.35177009527), (317960.94469078025, 245853.84258207353))
((317894.5420938695, 245824.35177009527), (317941.54076457256, 245885.65162972905))
((317942.0665525985, 245891.2323251177), (317932.9315142884, 245937.77282186068))
((317942.0665525985, 245891.2323251177), (317952.4183059603, 245902.62524526526))


Print out the node attributes:

In [8]:
print(G.nodes[(317995.5708697437, 245878.08877053304)])

{}


No nodes have *any* attributes

In [9]:
for i, node in enumerate(G.nodes()):
    if len(G.nodes[node]) > 0:
        print(node)

What does the edge data look like?

In [10]:
G.edges[(317995.5708697437, 245878.08877053304), (317973.5650240276, 245907.60584958008)]

{'Json': '{ "type": "LineString", "coordinates": [...] }',
 'ShpName': 'connections',
 'Wkb': b'...',
 'Wkt': 'LINESTRING (...)',
 'cat': 1,
 'func_class': 3,
 'length': 40,
 'link_id': ... ,
 'trav_dir': 'T',
 'u00_00': 48,
 'u00_15': 48,
 ...
 'urban': 'Y'}

Print out the 8am speed for the first 10 edges:

In [11]:
for i, (edge, speed) in enumerate(nx.get_edge_attributes(G,'u08_00').items()):
    if i == 10:
        break
    print(speed)

34
33
54
30
27
39
38
33
36
35


What does the graph structure of the first Weakly Connected Component look like?

In [12]:
for i, graph in enumerate(nx.weakly_connected_component_subgraphs(G)):
    print(nx.info(graph))
    if i == 0:
        break

Name: 
Type: DiGraph
Number of nodes: 84918
Number of edges: 95694
Average in degree:   1.1269
Average out degree:   1.1269


### Calculated Attributes

Calculate the travel time (in hours) for every road at 5:15pm

In [13]:
d = {}
for (n1, n2) in G.edges():
    e = G[n1][n2]
    d[(n1, n2)] = (e['length'] / 1000.0) / (e['u17_15'] * 1.0) # Convert 'length' from metres to km
nx.set_edge_attributes(G, d, 'time_1715')

In [14]:
print('Length (m): ', G[n1][n2]['length'])
print('Speed (km/h): ', G[n1][n2]['u17_15'])
print('Time (secs): ', G[n1][n2]['time_1715'] * 60 * 60)

Length (m):  12
Speed (km/h):  17
Time (secs):  2.5411764705882356


### Network Algorithms

Calculate the **Edge Betweeness Centrality** for each edge by sampling 1000 other edges weighted by travel time.

> For a given edge $e$, it is the sum over all node pairs of the fraction of all-pairs shortest paths that pass through $e$

In [15]:
%time bc = nx.edge_betweenness_centrality(G, k=1000, normalized=False, weight='time_1715', seed=999)
len(bc)

CPU times: user 4min 12s, sys: 1.79 s, total: 4min 14s
Wall time: 4min 13s


95867

Calculate the **Page Rank** for each node with edges weighted by travel time.

> A ranking of the nodes in the graph based on the structure of the incoming links.

In [16]:
%time pr = nx.pagerank(G, weight = 'time_1715')
pr_dict = {node: 0 for node in G.nodes} # Create dummy dictionary with all nodes as zeroes
pr_dict.update(pr)

CPU times: user 6.2 s, sys: 1.28 s, total: 7.49 s
Wall time: 7.75 s


Calculate the **Shortest Path Lengths** from Dublin's *The Spire* tourist attraction to every other point in Dublin.

In [17]:
# Find geographical coordinates of The Spire node as a tuple
approx_spire = (315920.300778, 234593.104858) # Estimate Coordinates from GRASS
for node in G.nodes():
    x, y = node
    if abs(approx_spire[0] - x) + abs(approx_spire[1] - y) < 1:
        spire = node # Find nearest node to within Manhattan Distance of 1
        break

In [18]:
%time path_lengths = nx.shortest_path_length(G, source = spire, target = None, weight = 'time_1715')

CPU times: user 31.5 ms, sys: 26 ms, total: 57.5 ms
Wall time: 56.3 ms


If the graph were strongly connected we would expect every node to be reachable from *The Spire*...

In [19]:
pl_dict = {node: 0 for node in G.nodes} # Create dummy dictionary with all nodes as zeroes
pl_dict.update(path_lengths)
len(path_lengths)

6086

Instead, only 6086 of the 85119 nodes are reachable

### Update Graph
Save the new metrics to graph structure.

In [20]:
nx.set_node_attributes(G, pl_dict, 'spire_time')
nx.set_node_attributes(G, pr_dict, 'page_rank')
nx.set_edge_attributes(G, bc, 'b_c')

### Output to .shp
Output our new metrics to a directory of shapefiles for further geospatial analysis

In [21]:
nx.write_shp(G, '../6_python_data_output/networkx_shp')

### Visualising the Graph
Use a dedicated Graph visualisation package…

In [22]:
# For each node, set 'label' attribute as the minimum func_class of all incident edges
for node in G.nodes():
    func_classes = [data['func_class'] for u, v, data in G.edges(node, data=True)]
    if len(func_classes) == 0:
        label = '5'
    else:
        label = str(min(func_classes))
    G.nodes[node]['label'] = label

In [23]:
# Remove binary data
for i, (u, v, data) in enumerate(G.edges(data=True)):
    for a in ['Wkb', 'Wkt', 'Json', 'ShpName']:
        if a in G[u][v]:
            del G[u][v][a]

In [24]:
# Get a subgraph of 100 nodes
H = G.subgraph([node for i, node in enumerate(G.nodes()) if i < 100])

Push subgraph to a local Docker Neo4j graph database

In [25]:
import neonx # from github.com/ruaridhw/neonx/tree/labels-auth
graph_db = 'http://localhost:7474/db/data/'
login = 'neo4j'
pw = 'focused_leavitt'
relationship = 'ROAD_TO'

results = neonx.write_to_neo(graph_db, H, relationship, server_login = login, server_pwd = pw)

Directed graph of 100 random nodes with edge road speed data preserved.

Each node is coloured using the minimum Function Class of the incident edges

<center><img src="../8_plots/graph.png" alt="Drawing" style="width: 400px;"/></center>